# Maven Reward Challenge
I’ll play the role of a Sr. Marketing Analyst at Maven Cafe.

Over a 30-day period, Maven Cafe tested different types of offers with Rewards members.

Now that the 30-day period for the test has concluded, my task is to identify key customer segments and develop a data-driven strategy for future promotional messaging & targeting.

Data that simulates the behavior of Cafe Rewards members over a 30-day period, including their transactions and responses to promotional offers.

The data is contained in three files: one with details on each offer, another with demographic information on each customer, and a third with the activity for each customer during the period.

The activities are divided into offer received, offer viewed, offer accepted, and transaction. For a transaction to be attributed to an offer, it must occur at the same time as when the offer was "completed" by the customer.

I have already cleaned the raw datasets. Cleaned offer, cleaned event, and cleaned_customer

## Objectives: 

The goal of this project was to answer three main questions:

** Which customers respond best to offers?
** What types of offers work best?
** How should we reach them to maximize impact?

Insights from this test will guide future promotional campaigns to bring in more revenue and improve customer loyalty.


## Part 1: Setup and Data Loading

### 1.1: Import Libraries
Import all packages. Group them by function to keep the setup clean and explain their purpose.

In [3]:
# Core libraries for data manipulation and numerical operations
import pandas as pd
import numpy as np

# Libraries for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# --- Configuration ---
# Set a consistent visual style for all plots
sns.set_style('whitegrid')
# Suppress routine warnings (e.g., from library versions) to keep output clean
import warnings
warnings.filterwarnings('ignore')

#import a script for data cleaning
from initial_report import *

### 1.2: Load Data
Load the three CSV files for immediate inspection

In [4]:
# Load DataFrames
df_customer = pd.read_csv('cleaned_customer_data.csv')
df_event = pd.read_csv('cleaned_events.csv')
df_offer = pd.read_csv('cleaned_offers.csv')

### 1.3: Data Overview Function

I already created a data overview function (initial_report) to make the process easier for data cleaning.

Now run the initial_report function for all three data set

In [7]:
# --- Inspect Customer Data ---
initial_report(df_customer)

 *** DATA CLEANING CHECKLIST ***
----------------------------------------
*** Structure:
- Total Rows: 14825
- Total Columns: 5
- Column Names: ['customer_id', 'became_member_on', 'gender', 'age', 'income']

*** Data Types:
  customer_id: object
  became_member_on: object
  gender: object
  age: int64
  income: float64

*** Mixed Data Types:

*** Distinct Values per Column:
  customer_id: 14825
  became_member_on: 1707
  gender: 3
  age: 84
  income: 91

*** Null Values and Percentages:


*** Duplicates: 0

*** Negative or Zero Values:

*** Basic Statistics:
                age         income
count  14825.000000   14825.000000
mean      54.393524   65404.991568
std       17.383705   21598.299410
min       18.000000   30000.000000
25%       42.000000   49000.000000
50%       55.000000   64000.000000
75%       66.000000   80000.000000
max      101.000000  120000.000000

*** Category Description:
                             customer_id became_member_on gender
count                       

In [8]:
# --- Inspect Event Data ---
initial_report(df_event)

 *** DATA CLEANING CHECKLIST ***
----------------------------------------
*** Structure:
- Total Rows: 306137
- Total Columns: 6
- Column Names: ['customer_id', 'event', 'time', 'offer_id', 'amount', 'reward']

*** Data Types:
  customer_id: object
  event: object
  time: int64
  offer_id: object
  amount: float64
  reward: float64

*** Mixed Data Types:
  offer_id:
    - str: 167184
    - float: 138953

*** Distinct Values per Column:
  customer_id: 17000
  event: 4
  time: 120
  offer_id: 10
  amount: 5103
  reward: 4

*** Null Values and Percentages:
  offer_id: Missing Values: 138953, Pct: 45.389%
  amount: Missing Values: 167184, Pct: 54.611%
  reward: Missing Values: 272955, Pct: 89.161%


*** Duplicates: 0

*** Negative or Zero Values:
  time: 15561

*** Basic Statistics:
                time         amount        reward
count  306137.000000  138953.000000  33182.000000
mean      366.185015      12.777356      4.902628
std       200.348174      30.250529      2.887201
min       

In [9]:
# --- Inspect Offer Data ---
initial_report(df_offer)

 *** DATA CLEANING CHECKLIST ***
----------------------------------------
*** Structure:
- Total Rows: 10
- Total Columns: 6
- Column Names: ['offer_id', 'offer_type', 'difficulty', 'reward', 'duration', 'channels']

*** Data Types:
  offer_id: object
  offer_type: object
  difficulty: int64
  reward: int64
  duration: int64
  channels: object

*** Mixed Data Types:

*** Distinct Values per Column:
  offer_id: 10
  offer_type: 3
  difficulty: 5
  reward: 5
  duration: 5
  channels: 4

*** Null Values and Percentages:


*** Duplicates: 0

*** Negative or Zero Values:
  difficulty: 2
  reward: 2

*** Basic Statistics:
       difficulty     reward   duration
count   10.000000  10.000000  10.000000
mean     7.700000   4.200000   6.500000
std      5.831905   3.583915   2.321398
min      0.000000   0.000000   3.000000
25%      5.000000   2.000000   5.000000
50%      8.500000   4.000000   7.000000
75%     10.000000   5.000000   7.000000
max     20.000000  10.000000  10.000000

*** Category De

### 1.5: Initial Findings & Cleaning Plan

Based on the overview:

1.  **`df_customer`**:
    * `became_member_on`: Is an `object` (string). It **must** be converted to `datetime` to analyze membership tenure.
    * `gender`: Contains 3 values ('M', 'F', 'O').
    * `age` & `income`: Are numerical with no missing values. I will bin these later to analyze cohorts. 

2.  **`df_event`**:
    * `event`: Has 4 unique values as expected ('offer received', 'offer viewed', 'offer completed', 'transaction').
    * `offer_id`, `amount`, `reward`: Have significant missing values. This is **expected** and not an error.
        * `offer_id` is null only for 'transaction' events.
        * `amount` is null only for 'offer' events.
        * `reward` is null for all except 'offer completed' events.
    * This structure confirms a central challenge: `offer_id` and `amount` are in separate rows.

3.  **`df_offer`**:
    * `duration`: This is in **days**. The `df_event['time']` column is in **hours**. These units are incompatible. **Need** to standardize them. **Need** to convert `duration` to hours.
    * `channels`: Is an `object` (a string that looks like a list). This is unusable for analysis and must be cleaned for channel effectiveness.

**Conclusion:** The data is structurally sound, but the `duration`/`time` unit mismatch is a critical flaw that must be fixed. The `became_member_on` conversion is also mandatory.

In [3]:
#get initial report
initial_report(df_customer)

 *** DATA CLEANING CHECKLIST ***
----------------------------------------
*** Structure:
- Total Rows: 14825
- Total Columns: 5
- Column Names: ['customer_id', 'became_member_on', 'gender', 'age', 'income']

*** Data Types:
  customer_id: object
  became_member_on: object
  gender: object
  age: int64
  income: float64

*** Mixed Data Types:

*** Distinct Values per Column:
  customer_id: 14825
  became_member_on: 1707
  gender: 3
  age: 84
  income: 91

*** Null Values and Percentages:


*** Duplicates: 0

*** Negative or Zero Values:

*** Basic Statistics:
                age         income
count  14825.000000   14825.000000
mean      54.393524   65404.991568
std       17.383705   21598.299410
min       18.000000   30000.000000
25%       42.000000   49000.000000
50%       55.000000   64000.000000
75%       66.000000   80000.000000
max      101.000000  120000.000000

*** Category Description:
                             customer_id became_member_on gender
count                       

Problems:
1. became member on column is object type- [change to datetype]

In [4]:
# Convert 'became_member_on' to datetime
df_customer['became_member_on'] = pd.to_datetime(df_customer['became_member_on'])
print(df_customer['became_member_on'].dtype)

datetime64[ns]


In [5]:
# Create age group column
df_customer['age_group'] = np.select(
    [
        (df_customer['age'] >= 18) & (df_customer['age'] <= 34),
        (df_customer['age'] >= 35) & (df_customer['age'] <= 49),
        (df_customer['age'] >= 50) & (df_customer['age'] <= 64),
        (df_customer['age'] >= 65) & (df_customer['age'] <= 79),
        (df_customer['age'] >= 80) & (df_customer['age'] <= 110)
    ],
    [
        'Young Adult',
        'Middle Age Adult',
        'Older Adult',
        'Senior',
        'Elderly'
    ],
    default='Unknown'
)

#print
df_customer.head()

,customer_id,became_member_on,gender,age,income,age_group
0,0610b486422d4921ae7d2bf64640c50b,2017-07-15,F,55,112000.0,Older Adult
1,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,F,75,100000.0,Senior
2,e2127556f4f64592b11af22de27a7932,2018-04-26,M,68,70000.0,Senior
3,389bc3fa690240e798340f5a15918d5c,2018-02-09,M,65,53000.0,Senior
4,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,M,58,51000.0,Older Adult


In [6]:
#get value counts for age group
df_customer.age_group.value_counts()

age_group
Older Adult         5150
Senior              3164
Middle Age Adult    3153
Young Adult         2256
Elderly             1102
Name: count, dtype: int64

In [7]:
#create income group

bins = [0, 44000, 84000, float('inf')]
labels = ['Low Income', 'Middle Income', 'High Income']

df_customer['income_group'] = pd.cut(df_customer['income'], bins=bins, labels=labels, right=True)

#check dataframe
df_customer.head()

,customer_id,became_member_on,gender,age,income,age_group,income_group
0,0610b486422d4921ae7d2bf64640c50b,2017-07-15,F,55,112000.0,Older Adult,High Income
1,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,F,75,100000.0,Senior,High Income
2,e2127556f4f64592b11af22de27a7932,2018-04-26,M,68,70000.0,Senior,Middle Income
3,389bc3fa690240e798340f5a15918d5c,2018-02-09,M,65,53000.0,Senior,Middle Income
4,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,M,58,51000.0,Older Adult,Middle Income


In [8]:
#get value counts for income group
df_customer.income_group.value_counts()

income_group
Middle Income    8941
High Income      3015
Low Income       2869
Name: count, dtype: int64

In [9]:
#get gender percentage
gender_percent = (
    df_customer.groupby('gender').size() / len(df_customer) * 100
)
gender_percent=round(gender_percent,2)
print(gender_percent)


gender
F    41.34
M    57.23
O     1.43
dtype: float64


In [10]:
#create group for membership year
df_customer['membership_year'] = df_customer['became_member_on'].astype(str).str[:4].astype(int)
df_customer.head()

,customer_id,became_member_on,gender,age,income,age_group,income_group,membership_year
0,0610b486422d4921ae7d2bf64640c50b,2017-07-15,F,55,112000.0,Older Adult,High Income,2017
1,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,F,75,100000.0,Senior,High Income,2017
2,e2127556f4f64592b11af22de27a7932,2018-04-26,M,68,70000.0,Senior,Middle Income,2018
3,389bc3fa690240e798340f5a15918d5c,2018-02-09,M,65,53000.0,Senior,Middle Income,2018
4,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,M,58,51000.0,Older Adult,Middle Income,2017


In [11]:
#get value counts for membership year
df_customer.membership_year.value_counts()

membership_year
2017    5599
2018    3669
2016    3024
2015    1597
2014     662
2013     274
Name: count, dtype: int64

In [12]:
#drop unnecessary columns
df_customer_bins = df_customer.drop(columns=['became_member_on', 'age','income'])
df_customer_bins.head()

,customer_id,gender,age_group,income_group,membership_year
0,0610b486422d4921ae7d2bf64640c50b,F,Older Adult,High Income,2017
1,78afa995795e4d85b5d9ceeca43f5fef,F,Senior,High Income,2017
2,e2127556f4f64592b11af22de27a7932,M,Senior,Middle Income,2018
3,389bc3fa690240e798340f5a15918d5c,M,Senior,Middle Income,2018
4,2eeac8d8feae4a8cad5a6af0499a211d,M,Older Adult,Middle Income,2017


In [13]:
#import event data
df_event=pd.read_csv("cleaned_events.csv")
df_event.head()

,customer_id,event,time,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


In [14]:
#merge df_customer_bins and df_event
df_customer_events = pd.merge(df_customer_bins, df_event, on='customer_id', how='left')
df_customer_events.head()

,customer_id,gender,age_group,income_group,membership_year,event,time,offer_id,amount,reward
0,0610b486422d4921ae7d2bf64640c50b,F,Older Adult,High Income,2017,transaction,18,NaN,21.51,NaN
1,0610b486422d4921ae7d2bf64640c50b,F,Older Adult,High Income,2017,transaction,144,NaN,32.28,NaN
2,0610b486422d4921ae7d2bf64640c50b,F,Older Adult,High Income,2017,offer received,408,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
3,0610b486422d4921ae7d2bf64640c50b,F,Older Adult,High Income,2017,offer received,504,3f207df678b143eea3cee63160fa8bed,NaN,NaN
4,0610b486422d4921ae7d2bf64640c50b,F,Older Adult,High Income,2017,transaction,528,NaN,23.22,NaN


In [15]:
initial_report(df_customer_events)

 *** DATA CLEANING CHECKLIST ***
----------------------------------------
*** Structure:
- Total Rows: 272388
- Total Columns: 10
- Column Names: ['customer_id', 'gender', 'age_group', 'income_group', 'membership_year', 'event', 'time', 'offer_id', 'amount', 'reward']

*** Data Types:
  customer_id: object
  gender: object
  age_group: object
  income_group: category
  membership_year: int64
  event: object
  time: int64
  offer_id: object
  amount: float64
  reward: float64

*** Mixed Data Types:
  offer_id:
    - str: 148431
    - float: 123957

*** Distinct Values per Column:
  customer_id: 14825
  gender: 3
  age_group: 5
  income_group: 3
  membership_year: 6
  event: 4
  time: 120
  offer_id: 10
  amount: 5036
  reward: 4

*** Null Values and Percentages:
  offer_id: Missing Values: 123957, Pct: 45.508%
  amount: Missing Values: 148431, Pct: 54.492%
  reward: Missing Values: 240318, Pct: 88.226%


*** Duplicates: 0

*** Negative or Zero Values:
  time: 13545

*** Basic Statistics

In [16]:
df_customer_events.event.value_counts()

event
transaction        123957
offer received      66501
offer viewed        49860
offer completed     32070
Name: count, dtype: int64

In [17]:
#percentage of offer_viewed and completed

# Calculate the value counts and get the total sum
event_counts = df_customer_events['event'].value_counts()

# Get the count for 'offer completed' and 'offer viewed'
offer_received_count = event_counts.get('offer received', 0)
offer_completed_count = event_counts.get('offer completed', 0)
offer_viewed_count = event_counts.get('offer viewed', 0)

# Calculate the percentages
offer_completed_percentage = (offer_completed_count / offer_received_count) * 100
offer_viewed_percentage = (offer_viewed_count / offer_received_count) * 100
offer_completed_after_view=(offer_completed_count / offer_viewed_count) * 100

# Print the results
print(f"Offer Viewed after offer received: {offer_viewed_percentage:.2f}%")
print(f"Offer Completed after offer received: {offer_completed_percentage:.2f}%")
print(f"Offer Completed after offer viewed: {offer_completed_after_view:.2f}%")


Offer Viewed after offer received: 74.98%
Offer Completed after offer received: 48.22%
Offer Completed after offer viewed: 64.32%


In [18]:
#total_transaction_amount
total_transaction_amount = df_customer_events['amount'].sum()
print(total_transaction_amount)

1734942.4


In [19]:
# Average transaction_amount
avg_transaction_amount = df_customer_events['amount'].mean()
print(avg_transaction_amount)


13.996324531894123


In [20]:
#get average spend per customer
distinct_customers_count = df_customer_events['customer_id'].nunique()
#get average
avg_spend_per_customer=total_transaction_amount/distinct_customers_count
print(avg_spend_per_customer)


117.02815514333895


In [21]:
#total spend based on age group
total_spend_age_group=df_customer_events.groupby("age_group")["amount"].sum().sort_values(ascending=False)
print(total_spend_age_group)

age_group
Older Adult         652728.13
Senior              401065.92
Middle Age Adult    343178.68
Young Adult         190943.30
Elderly             147026.37
Name: amount, dtype: float64


In [22]:
#average spend by age group
age_group_count= df_customer_events.groupby("age_group")["customer_id"].nunique()
avg_spend_age_group=total_spend_age_group/age_group_count.sort_values(ascending=False)
print(avg_spend_age_group)

age_group
Older Adult         126.743326
Senior              126.759140
Middle Age Adult    108.841954
Young Adult          84.637988
Elderly             133.417759
dtype: float64


In [23]:
#no. of transaction based on age_group
df_transaction=df_customer_events[df_customer_events.event=="transaction"]
number_of_trans_age_group=df_transaction.groupby("age_group")["event"].size().sort_values(ascending=False)
print(number_of_trans_age_group)

age_group
Older Adult         39988
Middle Age Adult    27618
Senior              24565
Young Adult         23183
Elderly              8603
Name: event, dtype: int64


In [24]:
#average transaction amount by age group
avg_transaction_amount_age_group=df_customer_events.groupby("age_group")["amount"].mean()
print(avg_transaction_amount_age_group)

age_group
Elderly             17.090128
Middle Age Adult    12.425906
Older Adult         16.323100
Senior              16.326722
Young Adult          8.236350
Name: amount, dtype: float64


In [25]:
#total spend based on membership year
total_spend_mem_year=df_customer_events.groupby("membership_year")["amount"].sum().sort_values(ascending=False)
print(total_spend_mem_year)

membership_year
2017    658287.36
2016    519551.67
2015    245725.55
2018    232690.24
2014     55311.60
2013     23375.98
Name: amount, dtype: float64


In [26]:
#total transaction based on income_group
total_spend_income_group=df_customer_events.groupby("income_group")["amount"].sum().sort_values(ascending=False)
print(total_spend_income_group)

income_group
Middle Income    1003664.00
High Income       547875.48
Low Income        183402.92
Name: amount, dtype: float64


In [27]:
#average transaciton amount by income group
income_group_count= df_customer_events.groupby("income_group")["customer_id"].nunique()
avg_spend_income_group=total_spend_income_group/income_group_count.sort_values(ascending=False)
print(avg_spend_income_group)

income_group
Middle Income    112.254110
High Income      181.716577
Low Income        63.925730
dtype: float64


In [28]:
#no. of transaction based on income_group
number_of_trans_income_group=df_transaction.groupby("income_group")["event"].size().sort_values(ascending=False)
print(number_of_trans_income_group)

income_group
Middle Income    76169
Low Income       28905
High Income      18883
Name: event, dtype: int64


In [29]:
#average transaction value by income group
avg_transaction_value_income_group=df_customer_events.groupby("income_group")["amount"].mean()
print(avg_transaction_value_income_group)

income_group
Low Income        6.345024
Middle Income    13.176804
High Income      29.014218
Name: amount, dtype: float64


In [30]:
#total spending by gender
total_transaction_by_gender=df_customer_events.groupby("gender")["amount"].sum()
print(total_transaction_by_gender)

gender
F    863695.00
M    844890.86
O     26356.54
Name: amount, dtype: float64


In [31]:
#average transaciton amount by gender
gender_count= df_customer_events.groupby("gender")["customer_id"].nunique()
avg_trans_gender=total_transaction_by_gender/gender_count.sort_values(ascending=False)
print(avg_trans_gender)

gender
F    140.919400
M     99.586381
O    124.323302
dtype: float64


In [32]:
#no. of transaction based on gender
number_of_trans_gender=df_transaction.groupby("gender")["event"].size().sort_values(ascending=False)
print(number_of_trans_gender)

gender
M    72794
F    49382
O     1781
Name: event, dtype: int64


In [33]:
#average transaction value by gender
avg_transaction_value_gender=df_customer_events.groupby("gender")["amount"].mean()
print(avg_transaction_value_gender)

gender
F    17.490077
M    11.606600
O    14.798731
Name: amount, dtype: float64


In [34]:
df_customer_events.head()

,customer_id,gender,age_group,income_group,membership_year,event,time,offer_id,amount,reward
0,0610b486422d4921ae7d2bf64640c50b,F,Older Adult,High Income,2017,transaction,18,NaN,21.51,NaN
1,0610b486422d4921ae7d2bf64640c50b,F,Older Adult,High Income,2017,transaction,144,NaN,32.28,NaN
2,0610b486422d4921ae7d2bf64640c50b,F,Older Adult,High Income,2017,offer received,408,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
3,0610b486422d4921ae7d2bf64640c50b,F,Older Adult,High Income,2017,offer received,504,3f207df678b143eea3cee63160fa8bed,NaN,NaN
4,0610b486422d4921ae7d2bf64640c50b,F,Older Adult,High Income,2017,transaction,528,NaN,23.22,NaN


In [35]:
#sort by customer_id and time

df_sorted = df_customer_events.sort_values(by=["customer_id","offer_id", "time"], ascending=[True, True, True])

# Reset index for cleanliness
df_sorted = df_sorted.reset_index(drop=True)

df_sorted.head()


,customer_id,gender,age_group,income_group,membership_year,event,time,offer_id,amount,reward
0,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer received,576,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
1,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer completed,576,2906b810c7d4411798c6938adc9daaa5,NaN,2.0
2,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer received,336,3f207df678b143eea3cee63160fa8bed,NaN,NaN
3,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer viewed,372,3f207df678b143eea3cee63160fa8bed,NaN,NaN
4,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer received,168,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN


In [36]:
#check status from one customer_id
df_sorted[df_sorted.customer_id=="e2127556f4f64592b11af22de27a7932"]

,customer_id,gender,age_group,income_group,membership_year,event,time,offer_id,amount,reward
240532,e2127556f4f64592b11af22de27a7932,M,Senior,Middle Income,2018,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
240533,e2127556f4f64592b11af22de27a7932,M,Senior,Middle Income,2018,offer viewed,18,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
240534,e2127556f4f64592b11af22de27a7932,M,Senior,Middle Income,2018,offer received,336,3f207df678b143eea3cee63160fa8bed,NaN,NaN
240535,e2127556f4f64592b11af22de27a7932,M,Senior,Middle Income,2018,offer received,408,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
240536,e2127556f4f64592b11af22de27a7932,M,Senior,Middle Income,2018,offer viewed,420,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
240537,e2127556f4f64592b11af22de27a7932,M,Senior,Middle Income,2018,offer completed,522,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,5.0
240538,e2127556f4f64592b11af22de27a7932,M,Senior,Middle Income,2018,offer received,504,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
240539,e2127556f4f64592b11af22de27a7932,M,Senior,Middle Income,2018,offer viewed,522,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
240540,e2127556f4f64592b11af22de27a7932,M,Senior,Middle Income,2018,offer completed,522,fafdcd668e3743c1bb461111dcafc2a4,NaN,2.0
240541,e2127556f4f64592b11af22de27a7932,M,Senior,Middle Income,2018,transaction,288,NaN,17.88,NaN


We can see that at time 522, one transaction happen and two offer_completed. So, it not possible to associate for what offer, the transaction was actually made. One person can receive multiple offers without even vieweing or completing the first offer. Transactions are not associated with offer_id. For a one single time, one transaction and multiple offer completion can occur. Which actually made the dataset complex to link any offer 
completion with transaction. However, we can try the dataset logic. 1. Check duration after offer receive. 2. Offer completion and transaction time should be same.

In [37]:
#get df_received
df_received=df_sorted[df_sorted.event=="offer received"]
df_received.rename(columns={'time': 'rcv_time'}, inplace=True)
df_received = df_received[['customer_id', 'rcv_time', 'offer_id']]
print(df_received.shape)
df_received.head()

(66501, 3)


,customer_id,rcv_time,offer_id
0,0009655768c64bdeb2e877511632db8f,576,2906b810c7d4411798c6938adc9daaa5
2,0009655768c64bdeb2e877511632db8f,336,3f207df678b143eea3cee63160fa8bed
4,0009655768c64bdeb2e877511632db8f,168,5a8bc65990b245e5a138643cd4eb9837
6,0009655768c64bdeb2e877511632db8f,408,f19421c1d4aa40978ebb69ca19b0e20d
9,0009655768c64bdeb2e877511632db8f,504,fafdcd668e3743c1bb461111dcafc2a4


In [38]:
#get df_viewed
df_viewed=df_sorted[df_sorted.event=="offer viewed"]
df_viewed.rename(columns={'time': 'view_time'}, inplace=True)
df_viewed = df_viewed[['customer_id', 'view_time', 'offer_id']]
print(df_viewed.shape)
df_viewed.head()

(49860, 3)


,customer_id,view_time,offer_id
3,0009655768c64bdeb2e877511632db8f,372,3f207df678b143eea3cee63160fa8bed
5,0009655768c64bdeb2e877511632db8f,192,5a8bc65990b245e5a138643cd4eb9837
8,0009655768c64bdeb2e877511632db8f,456,f19421c1d4aa40978ebb69ca19b0e20d
11,0009655768c64bdeb2e877511632db8f,540,fafdcd668e3743c1bb461111dcafc2a4
21,0011e0d4e6b944f998e987f904e8c1e5,432,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [39]:
#merge df_received and viewed
df_rcv_view=pd.merge(df_received,df_viewed, on=["customer_id", "offer_id"],how="left")
print(df_rcv_view.shape)
print(df_rcv_view.duplicated().sum()) 
df_rcv_view.head()

(82834, 4)
0


,customer_id,rcv_time,offer_id,view_time
0,0009655768c64bdeb2e877511632db8f,576,2906b810c7d4411798c6938adc9daaa5,NaN
1,0009655768c64bdeb2e877511632db8f,336,3f207df678b143eea3cee63160fa8bed,372.0
2,0009655768c64bdeb2e877511632db8f,168,5a8bc65990b245e5a138643cd4eb9837,192.0
3,0009655768c64bdeb2e877511632db8f,408,f19421c1d4aa40978ebb69ca19b0e20d,456.0
4,0009655768c64bdeb2e877511632db8f,504,fafdcd668e3743c1bb461111dcafc2a4,540.0


In [40]:
# can one customer can rcv and view same offer at multiple times?
df_rcv_view[df_rcv_view.duplicated(subset=['customer_id', 'offer_id'], keep=False)]

,customer_id,rcv_time,offer_id,view_time
13,0020c2b971eb4e9188eac86d93036a77,0,fafdcd668e3743c1bb461111dcafc2a4,12.0
14,0020c2b971eb4e9188eac86d93036a77,336,fafdcd668e3743c1bb461111dcafc2a4,12.0
22,003d66b6608740288d6cc97a6903f4f0,168,fafdcd668e3743c1bb461111dcafc2a4,300.0
23,003d66b6608740288d6cc97a6903f4f0,168,fafdcd668e3743c1bb461111dcafc2a4,420.0
24,003d66b6608740288d6cc97a6903f4f0,408,fafdcd668e3743c1bb461111dcafc2a4,300.0
...,...,...,...,...
82827,ffff82501cea40309d5fdd7edcca4a07,408,2906b810c7d4411798c6938adc9daaa5,414.0
82828,ffff82501cea40309d5fdd7edcca4a07,408,2906b810c7d4411798c6938adc9daaa5,582.0
82829,ffff82501cea40309d5fdd7edcca4a07,576,2906b810c7d4411798c6938adc9daaa5,354.0
82830,ffff82501cea40309d5fdd7edcca4a07,576,2906b810c7d4411798c6938adc9daaa5,414.0


One customer id can receive and view same offers at multiple times. And some cases, we can see view time is before rcv time, which is not logical. So need to get rid of those data

In [41]:
#get valid rows based on logic that view time should be after rcv_time
df_rcv_view['status'] = np.where(
    df_rcv_view['rcv_time'] <= df_rcv_view['view_time'],
    'valid',
    'invalid'
)
print(df_rcv_view.shape)
df_rcv_view.head()

(82834, 5)


,customer_id,rcv_time,offer_id,view_time,status
0,0009655768c64bdeb2e877511632db8f,576,2906b810c7d4411798c6938adc9daaa5,NaN,invalid
1,0009655768c64bdeb2e877511632db8f,336,3f207df678b143eea3cee63160fa8bed,372.0,valid
2,0009655768c64bdeb2e877511632db8f,168,5a8bc65990b245e5a138643cd4eb9837,192.0,valid
3,0009655768c64bdeb2e877511632db8f,408,f19421c1d4aa40978ebb69ca19b0e20d,456.0,valid
4,0009655768c64bdeb2e877511632db8f,504,fafdcd668e3743c1bb461111dcafc2a4,540.0,valid


In [42]:
#take only the valid data
df_rcv_view=df_rcv_view[df_rcv_view.status=="valid"]
df_rcv_view = df_rcv_view.drop('status', axis=1)
print(df_rcv_view.shape)
df_rcv_view.head()

(59031, 4)


,customer_id,rcv_time,offer_id,view_time
1,0009655768c64bdeb2e877511632db8f,336,3f207df678b143eea3cee63160fa8bed,372.0
2,0009655768c64bdeb2e877511632db8f,168,5a8bc65990b245e5a138643cd4eb9837,192.0
3,0009655768c64bdeb2e877511632db8f,408,f19421c1d4aa40978ebb69ca19b0e20d,456.0
4,0009655768c64bdeb2e877511632db8f,504,fafdcd668e3743c1bb461111dcafc2a4,540.0
5,0011e0d4e6b944f998e987f904e8c1e5,408,0b1e1539f2cc45b7b9fa7c272da2e1d7,432.0


In [43]:
#get df_completed
df_completed=df_sorted[df_sorted.event=="offer completed"]
df_completed.rename(columns={'time': 'comp_time'}, inplace=True)
df_completed = df_completed[['customer_id', 'comp_time', 'offer_id']]
print(df_completed.shape)
df_completed.head()

(32070, 3)


,customer_id,comp_time,offer_id
1,0009655768c64bdeb2e877511632db8f,576,2906b810c7d4411798c6938adc9daaa5
7,0009655768c64bdeb2e877511632db8f,414,f19421c1d4aa40978ebb69ca19b0e20d
10,0009655768c64bdeb2e877511632db8f,528,fafdcd668e3743c1bb461111dcafc2a4
22,0011e0d4e6b944f998e987f904e8c1e5,576,0b1e1539f2cc45b7b9fa7c272da2e1d7
25,0011e0d4e6b944f998e987f904e8c1e5,252,2298d6c36e964ae4a3e7e9706d1fb8c2


In [44]:
#merge df_rcv_view and completed
df_time=pd.merge(df_rcv_view,df_completed, on=["customer_id", "offer_id"],how="left")
print(df_time.shape)
print(df_time.duplicated().sum()) 
df_time.head()

(71675, 5)
0


,customer_id,rcv_time,offer_id,view_time,comp_time
0,0009655768c64bdeb2e877511632db8f,336,3f207df678b143eea3cee63160fa8bed,372.0,NaN
1,0009655768c64bdeb2e877511632db8f,168,5a8bc65990b245e5a138643cd4eb9837,192.0,NaN
2,0009655768c64bdeb2e877511632db8f,408,f19421c1d4aa40978ebb69ca19b0e20d,456.0,414.0
3,0009655768c64bdeb2e877511632db8f,504,fafdcd668e3743c1bb461111dcafc2a4,540.0,528.0
4,0011e0d4e6b944f998e987f904e8c1e5,408,0b1e1539f2cc45b7b9fa7c272da2e1d7,432.0,576.0


 A customer can complete an offer without viewing it. It can happen in real life.

In [45]:
#avg diff between rcv_time and view time
average_difference_view = (df_time['view_time'] - df_time['rcv_time']).mean()
print(average_difference_view)

78.24142309033833


In [46]:
#avg diff between view_time and completed time
average_difference_comp = (df_time['comp_time'] - df_time['view_time']).mean()
print(average_difference_comp)

4.840387950414285


In [47]:
#avg diff between rcv_time and completed time
avg_diff_rcv_comp = (df_time['comp_time'] - df_time['rcv_time']).mean()
print(avg_diff_rcv_comp)

90.29327509795105


In [48]:
#import offer data
df_offer=pd.read_csv("cleaned_offers.csv")
df_offer.head()

,offer_id,offer_type,difficulty,reward,duration,channels
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,10,7,"['email', 'mobile', 'social']"
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,10,5,"['web', 'email', 'mobile', 'social']"
2,3f207df678b143eea3cee63160fa8bed,informational,0,0,4,"['web', 'email', 'mobile']"
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,5,7,"['web', 'email', 'mobile']"
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20,5,10,"['web', 'email']"


In [49]:
#merge df_time and df_offer to add duration
df_time_duration=pd.merge(df_time,df_offer, on="offer_id",how="left")
df_time_duration = df_time_duration[['customer_id', "offer_id","rcv_time","view_time", 'comp_time', 'duration']]
print(df_time_duration.shape)
print(df_time_duration.duplicated().sum()) 
df_time_duration.head()

(71675, 6)
0


,customer_id,offer_id,rcv_time,view_time,comp_time,duration
0,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,336,372.0,NaN,4
1,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,168,192.0,NaN,3
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,5
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504,540.0,528.0,10
4,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408,432.0,576.0,10


In [50]:
#convert duration from day to hours
df_time_duration.duration=df_time_duration["duration"]*24
df_time_duration.head()

,customer_id,offer_id,rcv_time,view_time,comp_time,duration
0,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,336,372.0,NaN,96
1,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,168,192.0,NaN,72
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,120
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504,540.0,528.0,240
4,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408,432.0,576.0,240


In [51]:
#a valid offer completion time should be lower or equal than rcv_time+duration
df_time_duration["valid_comp_time"]=df_time_duration.rcv_time+df_time_duration.duration
df_time_duration.head()

,customer_id,offer_id,rcv_time,view_time,comp_time,duration,valid_comp_time
0,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,336,372.0,NaN,96,432
1,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,168,192.0,NaN,72,240
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,120,528
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504,540.0,528.0,240,744
4,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408,432.0,576.0,240,648


In [52]:
#get valid status column
df_time_duration['status'] = np.where(
    df_time_duration['comp_time'] <= df_time_duration['valid_comp_time'],
    'valid',
    'invalid'
)
df_time_duration.head()

,customer_id,offer_id,rcv_time,view_time,comp_time,duration,valid_comp_time,status
0,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,336,372.0,NaN,96,432,invalid
1,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,168,192.0,NaN,72,240,invalid
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,120,528,valid
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504,540.0,528.0,240,744,valid
4,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408,432.0,576.0,240,648,valid


In [53]:
#check status from one customer_id
df_time_duration[df_time_duration.customer_id=="e2127556f4f64592b11af22de27a7932"]

,customer_id,offer_id,rcv_time,view_time,comp_time,duration,valid_comp_time,status
63227,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,0,18.0,NaN,168,168,invalid
63228,e2127556f4f64592b11af22de27a7932,9b98b8c7a33c4b65b9aebfe6a799e6d9,408,420.0,522.0,168,576,valid
63229,e2127556f4f64592b11af22de27a7932,fafdcd668e3743c1bb461111dcafc2a4,504,522.0,522.0,240,744,valid


In [54]:
#get valid data only
df_time_valid=df_time_duration[df_time_duration.status=="valid"]
print(df_time_valid.shape)
df_time_valid.head()

(37952, 8)


,customer_id,offer_id,rcv_time,view_time,comp_time,duration,valid_comp_time,status
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,120,528,valid
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504,540.0,528.0,240,744,valid
4,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408,432.0,576.0,240,648,valid
5,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,168,186.0,252.0,168,336,valid
8,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,504,516.0,576.0,168,672,valid


In [55]:
#delete unnecessary columns
df_time_valid = df_time_valid[['customer_id', "offer_id","rcv_time","view_time", 'comp_time']]
print(df_time_valid.shape)
df_time_valid.head()

(37952, 5)


,customer_id,offer_id,rcv_time,view_time,comp_time
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504,540.0,528.0
4,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408,432.0,576.0
5,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,168,186.0,252.0
8,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,504,516.0,576.0


total offer_completion was 32070, however, we got 37952 rows. That means one offer_completion time got attached to multiple rcv_time and view_time

In [56]:
#offer completion time should be higher than offer receive time
df_time_valid[df_time_valid.rcv_time>=df_time_valid.comp_time]

,customer_id,offer_id,rcv_time,view_time,comp_time
23,003d66b6608740288d6cc97a6903f4f0,fafdcd668e3743c1bb461111dcafc2a4,408,420.0,384.0
33,004c5799adbf42868b9cff0396190900,f19421c1d4aa40978ebb69ca19b0e20d,504,516.0,432.0
39,004c5799adbf42868b9cff0396190900,fafdcd668e3743c1bb461111dcafc2a4,576,648.0,222.0
40,004c5799adbf42868b9cff0396190900,fafdcd668e3743c1bb461111dcafc2a4,576,648.0,576.0
47,0056df74b63b4298809f0b375a304cf4,2298d6c36e964ae4a3e7e9706d1fb8c2,576,654.0,576.0
...,...,...,...,...,...
71667,ffff82501cea40309d5fdd7edcca4a07,2906b810c7d4411798c6938adc9daaa5,408,582.0,384.0
71670,ffff82501cea40309d5fdd7edcca4a07,2906b810c7d4411798c6938adc9daaa5,576,582.0,384.0
71671,ffff82501cea40309d5fdd7edcca4a07,2906b810c7d4411798c6938adc9daaa5,576,582.0,414.0
71672,ffff82501cea40309d5fdd7edcca4a07,2906b810c7d4411798c6938adc9daaa5,576,582.0,576.0


In [57]:
#get valid status column
df_time_valid['status'] = np.where(
    df_time_valid['rcv_time'] <= df_time_valid['comp_time'],
    'valid',
    'invalid'
)
df_time_valid.head()

,customer_id,offer_id,rcv_time,view_time,comp_time,status
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,valid
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504,540.0,528.0,valid
4,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408,432.0,576.0,valid
5,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,168,186.0,252.0,valid
8,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,504,516.0,576.0,valid


In [58]:
df_time_valid=df_time_valid[df_time_valid.status=="valid"]
df_time_valid=df_time_valid.drop("status",axis=1)
print(df_time_valid.shape)
df_time_valid.head()

(33133, 5)


,customer_id,offer_id,rcv_time,view_time,comp_time
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0
3,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504,540.0,528.0
4,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408,432.0,576.0
5,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,168,186.0,252.0
8,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,504,516.0,576.0


In [59]:
#add df_transaction to apply 2nd logic [the valid offer_completion will be when transaction time and completed time is same]
#get df_transaction
df_transactions=df_sorted[df_sorted.event=="transaction"]
df_transactions.rename(columns={'time': 'TT_time'}, inplace=True)
df_transactions = df_transactions[['customer_id', 'TT_time']]
print(df_transactions.shape)
df_transactions.head()

(123957, 2)


,customer_id,TT_time
12,0009655768c64bdeb2e877511632db8f,228
13,0009655768c64bdeb2e877511632db8f,414
14,0009655768c64bdeb2e877511632db8f,528
15,0009655768c64bdeb2e877511632db8f,552
16,0009655768c64bdeb2e877511632db8f,576


In [60]:
#merge df_transactions and df_time_valid
df_time_TT=pd.merge(df_time_valid,df_transactions, on="customer_id",how="left")
print(df_time_TT.shape)
print(df_time_TT.duplicated().sum()) 
df_time_TT.head()

(339464, 6)
0


,customer_id,offer_id,rcv_time,view_time,comp_time,TT_time
0,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,228
1,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,414
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,528
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,552
4,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,576


In [61]:
#get valid status column
df_time_TT['status'] = np.where(
    df_time_TT['comp_time'] == df_time_TT['TT_time'],
    'valid',
    'invalid'
)
df_time_TT.head()

,customer_id,offer_id,rcv_time,view_time,comp_time,TT_time,status
0,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,228,invalid
1,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,414,valid
2,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,528,invalid
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,552,invalid
4,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,576,invalid


In [62]:
#get valid data only
df_final_offers=df_time_TT[df_time_TT.status=="valid"]
print(df_final_offers.shape)
print(df_final_offers.duplicated().sum())
df_final_offers.head()

(33133, 7)
0


,customer_id,offer_id,rcv_time,view_time,comp_time,TT_time,status
1,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,414,valid
10,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504,540.0,528.0,528,valid
18,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408,432.0,576.0,576,valid
22,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,168,186.0,252.0,252,valid
28,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,504,516.0,576.0,576,valid


In [63]:
#check status from one customer_id
df_final_offers[df_final_offers.customer_id=="e2127556f4f64592b11af22de27a7932"]

,customer_id,offer_id,rcv_time,view_time,comp_time,TT_time,status
297212,e2127556f4f64592b11af22de27a7932,9b98b8c7a33c4b65b9aebfe6a799e6d9,408,420.0,522.0,522,valid
297215,e2127556f4f64592b11af22de27a7932,fafdcd668e3743c1bb461111dcafc2a4,504,522.0,522.0,522,valid


There was only one transaction at time 522. However, it became associated with two offer_id which can not happen.

In [64]:
#number or rows where tt_time is duplicated
df_final_offers[df_final_offers.duplicated(subset=['customer_id', 'offer_id'], keep=False)]

,customer_id,offer_id,rcv_time,view_time,comp_time,TT_time,status
89,003d66b6608740288d6cc97a6903f4f0,fafdcd668e3743c1bb461111dcafc2a4,168,300.0,384.0,384,valid
107,003d66b6608740288d6cc97a6903f4f0,fafdcd668e3743c1bb461111dcafc2a4,168,420.0,384.0,384,valid
131,003d66b6608740288d6cc97a6903f4f0,fafdcd668e3743c1bb461111dcafc2a4,408,420.0,504.0,504,valid
171,004c5799adbf42868b9cff0396190900,f19421c1d4aa40978ebb69ca19b0e20d,408,408.0,432.0,432,valid
183,004c5799adbf42868b9cff0396190900,f19421c1d4aa40978ebb69ca19b0e20d,408,516.0,432.0,432,valid
...,...,...,...,...,...,...,...
339368,ffff82501cea40309d5fdd7edcca4a07,2906b810c7d4411798c6938adc9daaa5,408,414.0,414.0,414,valid
339386,ffff82501cea40309d5fdd7edcca4a07,2906b810c7d4411798c6938adc9daaa5,408,414.0,576.0,576,valid
339398,ffff82501cea40309d5fdd7edcca4a07,2906b810c7d4411798c6938adc9daaa5,408,582.0,414.0,414,valid
339416,ffff82501cea40309d5fdd7edcca4a07,2906b810c7d4411798c6938adc9daaa5,408,582.0,576.0,576,valid


In [65]:
#get valid status column
df_final_offers['status'] = np.where(
    df_final_offers['comp_time'] >= df_final_offers["view_time"],
    'valid',
    'invalid'
)
df_final_offers.head()

,customer_id,offer_id,rcv_time,view_time,comp_time,TT_time,status
1,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,408,456.0,414.0,414,invalid
10,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,504,540.0,528.0,528,invalid
18,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408,432.0,576.0,576,valid
22,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,168,186.0,252.0,252,valid
28,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,504,516.0,576.0,576,valid


In [66]:
#get valid data only
df_final_offers=df_final_offers[df_final_offers.status=="valid"]
print(df_final_offers.shape)
print(df_final_offers.duplicated().sum())
df_final_offers.head()

(24172, 7)
0


,customer_id,offer_id,rcv_time,view_time,comp_time,TT_time,status
18,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,408,432.0,576.0,576,valid
22,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,168,186.0,252.0,252,valid
28,0011e0d4e6b944f998e987f904e8c1e5,9b98b8c7a33c4b65b9aebfe6a799e6d9,504,516.0,576.0,576,valid
35,0020c2b971eb4e9188eac86d93036a77,4d5c57ea9a6940dd891ad53e9dbe8da0,408,426.0,510.0,510,valid
39,0020c2b971eb4e9188eac86d93036a77,fafdcd668e3743c1bb461111dcafc2a4,0,12.0,54.0,54,valid


In [67]:
#check status from one customer_id
df_final_offers[df_final_offers.customer_id=="e2127556f4f64592b11af22de27a7932"]

,customer_id,offer_id,rcv_time,view_time,comp_time,TT_time,status
297212,e2127556f4f64592b11af22de27a7932,9b98b8c7a33c4b65b9aebfe6a799e6d9,408,420.0,522.0,522,valid
297215,e2127556f4f64592b11af22de27a7932,fafdcd668e3743c1bb461111dcafc2a4,504,522.0,522.0,522,valid


In [68]:
#get df_sorted
df_sorted.head()

,customer_id,gender,age_group,income_group,membership_year,event,time,offer_id,amount,reward
0,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer received,576,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
1,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer completed,576,2906b810c7d4411798c6938adc9daaa5,NaN,2.0
2,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer received,336,3f207df678b143eea3cee63160fa8bed,NaN,NaN
3,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer viewed,372,3f207df678b143eea3cee63160fa8bed,NaN,NaN
4,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer received,168,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN


In [69]:
#get df_completed
df_completed=df_sorted[df_sorted.event=="offer completed"]
print(df_completed.shape)
df_completed.head()

(32070, 10)


,customer_id,gender,age_group,income_group,membership_year,event,time,offer_id,amount,reward
1,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer completed,576,2906b810c7d4411798c6938adc9daaa5,NaN,2.0
7,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer completed,414,f19421c1d4aa40978ebb69ca19b0e20d,NaN,5.0
10,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer completed,528,fafdcd668e3743c1bb461111dcafc2a4,NaN,2.0
22,0011e0d4e6b944f998e987f904e8c1e5,O,Middle Age Adult,Middle Income,2018,offer completed,576,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,5.0
25,0011e0d4e6b944f998e987f904e8c1e5,O,Middle Age Adult,Middle Income,2018,offer completed,252,2298d6c36e964ae4a3e7e9706d1fb8c2,NaN,3.0


In [70]:
#get df_transaction
df_transaction=df_sorted[df_sorted.event=="transaction"]
print(df_transaction.shape)
df_transaction.head()

(123957, 10)


,customer_id,gender,age_group,income_group,membership_year,event,time,offer_id,amount,reward
12,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,transaction,228,NaN,22.16,NaN
13,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,transaction,414,NaN,8.57,NaN
14,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,transaction,528,NaN,14.11,NaN
15,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,transaction,552,NaN,13.56,NaN
16,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,transaction,576,NaN,10.27,NaN


In [71]:
#get rows where transaction time and completed time is same
matching_pairs = pd.merge(
    df_completed[['customer_id', 'time']].drop_duplicates(),
    df_transaction[['customer_id', 'time']].drop_duplicates(),
    on=['customer_id', 'time'],
    how='inner'
)
matching_pairs

,customer_id,time
0,0009655768c64bdeb2e877511632db8f,576
1,0009655768c64bdeb2e877511632db8f,414
2,0009655768c64bdeb2e877511632db8f,528
3,0011e0d4e6b944f998e987f904e8c1e5,576
4,0011e0d4e6b944f998e987f904e8c1e5,252
...,...,...
29576,ffff82501cea40309d5fdd7edcca4a07,384
29577,ffff82501cea40309d5fdd7edcca4a07,414
29578,ffff82501cea40309d5fdd7edcca4a07,576
29579,ffff82501cea40309d5fdd7edcca4a07,504


In [72]:
#filter df_transaction to get offer_related transaction
df_offer_tran = pd.merge(df_transaction, matching_pairs, on=['customer_id', 'time'], how='inner')
print(df_offer_tran.shape)
df_offer_tran.head()

(29581, 10)


,customer_id,gender,age_group,income_group,membership_year,event,time,offer_id,amount,reward
0,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,transaction,414,NaN,8.57,NaN
1,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,transaction,528,NaN,14.11,NaN
2,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,transaction,576,NaN,10.27,NaN
3,0011e0d4e6b944f998e987f904e8c1e5,O,Middle Age Adult,Middle Income,2018,transaction,252,NaN,11.93,NaN
4,0011e0d4e6b944f998e987f904e8c1e5,O,Middle Age Adult,Middle Income,2018,transaction,576,NaN,22.05,NaN


In [73]:
#total offer related transaction amount
df_offer_tran.amount.sum()

np.float64(606355.75)

In [74]:
#get gender percentage
gender_percent_trans = (
    df_offer_tran.groupby('gender').size() / len(df_offer_tran) * 100
)
gender_percent_trans=round(gender_percent_trans,2)
print(gender_percent_trans)

gender
F    47.58
M    50.87
O     1.56
dtype: float64


In [75]:
#transaction amount based on gender
offer_transaction_amount_gender=df_offer_tran.groupby("gender")["amount"].sum()
print(offer_transaction_amount_gender)

gender
F    312028.64
M    285536.88
O      8790.23
Name: amount, dtype: float64


In [76]:
#average transaciton amount by gender
gender_count= df_offer_tran.groupby("gender")["customer_id"].nunique()
avg_trans_gender=offer_transaction_amount_gender/gender_count.sort_values(ascending=False)
print(avg_trans_gender)

gender
F    57.879547
M    44.503878
O    49.107430
dtype: float64


In [77]:
#no. of transaction based on gender
number_of_trans_gender=df_offer_tran.groupby("gender")["event"].size().sort_values(ascending=False)
print(number_of_trans_gender)

gender
M    15047
F    14074
O      460
Name: event, dtype: int64


In [78]:
#average transaction value by gender
avg_transaction_value_gender=df_offer_tran.groupby("gender")["amount"].mean()
print(avg_transaction_value_gender)

gender
F    22.170573
M    18.976333
O    19.109196
Name: amount, dtype: float64


In [79]:
#transaction amount based on age_group
transaction_amount_age_group=df_offer_tran.groupby("age_group")["amount"].sum().sort_values(ascending=False)
print(transaction_amount_age_group)

age_group
Older Adult         242947.70
Senior              149380.68
Middle Age Adult    108934.62
Young Adult          52583.33
Elderly              52509.42
Name: amount, dtype: float64


In [80]:
#average transaciton amount by age_group
age_group_count= df_offer_tran.groupby("age_group")["customer_id"].nunique()
avg_trans_age_group=transaction_amount_age_group/age_group_count.sort_values(ascending=False)
print(avg_trans_age_group)

age_group
Older Adult         55.888590
Senior              57.146396
Middle Age Adult    43.889855
Young Adult         32.803075
Elderly             55.861085
dtype: float64


In [81]:
#no. of transaction based on age_group
number_of_trans_age_group=df_offer_tran.groupby("age_group")["event"].size().sort_values(ascending=False)
print(number_of_trans_age_group)

age_group
Older Adult         10906
Senior               6544
Middle Age Adult     6117
Young Adult          3648
Elderly              2366
Name: event, dtype: int64


In [82]:
#average transaction value by age_group
avg_transaction_value_age_group=df_offer_tran.groupby("age_group")["amount"].mean().sort_values(ascending=False)
print(avg_transaction_value_age_group)

age_group
Senior              22.827121
Older Adult         22.276518
Elderly             22.193331
Middle Age Adult    17.808504
Young Adult         14.414290
Name: amount, dtype: float64


In [83]:
#transaction amount based on membership year
transaction_amount_year=df_offer_tran.groupby("membership_year")["amount"].sum().sort_values(ascending=False)
print(transaction_amount_year)

membership_year
2017    229763.64
2016    173249.20
2018     94241.96
2015     82172.29
2014     19304.66
2013      7624.00
Name: amount, dtype: float64


In [84]:
#transaction amount based on income_group
transaction_amount_income_group=df_offer_tran.groupby("income_group")["amount"].sum().sort_values(ascending=False)
print(transaction_amount_income_group)

income_group
Middle Income    333084.10
High Income      220936.44
Low Income        52335.21
Name: amount, dtype: float64


In [85]:
#average transaciton amount by age_group
income_group_count= df_offer_tran.groupby("income_group")["customer_id"].nunique()
avg_trans_income_group=transaction_amount_income_group/income_group_count.sort_values(ascending=False)
print(avg_trans_income_group)

income_group
Middle Income    46.044249
High Income      78.429691
Low Income       27.046620
dtype: float64


In [86]:
#no. of transaction based on income_group
number_of_trans_income_group=df_offer_tran.groupby("income_group")["event"].size().sort_values(ascending=False)
print(number_of_trans_income_group)

income_group
Middle Income    17956
High Income       7452
Low Income        4173
Name: event, dtype: int64


In [87]:
#average transaction value by income_group
avg_transaction_value_income_group=df_offer_tran.groupby("income_group")["amount"].mean().sort_values(ascending=False)
print(avg_transaction_value_income_group)

income_group
High Income      29.647939
Middle Income    18.550017
Low Income       12.541387
Name: amount, dtype: float64


In [88]:
#get df_completed
df_completed.drop("amount",axis=1,inplace=True)
df_completed.head()

,customer_id,gender,age_group,income_group,membership_year,event,time,offer_id,reward
1,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer completed,576,2906b810c7d4411798c6938adc9daaa5,2.0
7,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer completed,414,f19421c1d4aa40978ebb69ca19b0e20d,5.0
10,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer completed,528,fafdcd668e3743c1bb461111dcafc2a4,2.0
22,0011e0d4e6b944f998e987f904e8c1e5,O,Middle Age Adult,Middle Income,2018,offer completed,576,0b1e1539f2cc45b7b9fa7c272da2e1d7,5.0
25,0011e0d4e6b944f998e987f904e8c1e5,O,Middle Age Adult,Middle Income,2018,offer completed,252,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0


In [89]:
#total reward
df_completed.reward.sum()

np.float64(158630.0)

In [90]:
#get gender percentage
gender_percent_compl = (
    df_completed.groupby('gender').size() / len(df_completed) * 100
)
gender_percent_compl=round(gender_percent_compl,2)
print(gender_percent_compl)

gender
F    47.76
M    50.70
O     1.54
dtype: float64


In [91]:
#offer completion based on genders
df_completed.gender.value_counts()

gender
M    16259
F    15316
O      495
Name: count, dtype: int64

In [92]:
#offer completion based on age_group
df_completed.age_group.value_counts()

age_group
Older Adult         11898
Senior               7127
Middle Age Adult     6558
Young Adult          3889
Elderly              2598
Name: count, dtype: int64

In [93]:
#offer completion based on membership year
df_completed.membership_year.value_counts()

membership_year
2017    11987
2016     8836
2018     5119
2015     4262
2014     1308
2013      558
Name: count, dtype: int64

In [94]:
#offer completion based on income_group
df_completed.income_group.value_counts()

income_group
Middle Income    19403
High Income       8222
Low Income        4445
Name: count, dtype: int64

In [95]:
#import offer data
df_offer=pd.read_csv("cleaned_offers.csv")
df_offer.head()

,offer_id,offer_type,difficulty,reward,duration,channels
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,10,7,"['email', 'mobile', 'social']"
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,10,5,"['web', 'email', 'mobile', 'social']"
2,3f207df678b143eea3cee63160fa8bed,informational,0,0,4,"['web', 'email', 'mobile']"
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,5,7,"['web', 'email', 'mobile']"
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20,5,10,"['web', 'email']"


In [96]:
# Convert the numerical columns to strings before concatenating
df_offer['difficulty'] = df_offer['difficulty'].astype(str)
df_offer['reward'] = df_offer['reward'].astype(str)
df_offer['duration'] = df_offer['duration'].astype(str)

# Create the new column by concatenating the strings with hyphens
df_offer['offer_key'] = (
    df_offer['offer_type'] + '-' + 
    df_offer['difficulty'] + '-' + 
    df_offer['reward'] + '-' + 
    df_offer['duration']
)

# Display the DataFrame with the new column
print("DataFrame with the new 'offer_key' column:")
print(df_offer[['offer_id', 'offer_key']].head())

DataFrame with the new 'offer_key' column:
                           offer_id            offer_key
0  ae264e3637204a6fb9bb56bc8210ddfd         bogo-10-10-7
1  4d5c57ea9a6940dd891ad53e9dbe8da0         bogo-10-10-5
2  3f207df678b143eea3cee63160fa8bed  informational-0-0-4
3  9b98b8c7a33c4b65b9aebfe6a799e6d9           bogo-5-5-7
4  0b1e1539f2cc45b7b9fa7c272da2e1d7     discount-20-5-10


In [143]:
#load df_offer
df_offer

,offer_id,offer_type,difficulty,reward,duration,channels,offer_key
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,10,7,"['email', 'mobile', 'social']",bogo-10-10-7
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,10,5,"['web', 'email', 'mobile', 'social']",bogo-10-10-5
2,3f207df678b143eea3cee63160fa8bed,informational,0,0,4,"['web', 'email', 'mobile']",informational-0-0-4
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,5,7,"['web', 'email', 'mobile']",bogo-5-5-7
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20,5,10,"['web', 'email']",discount-20-5-10
5,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,7,3,7,"['web', 'email', 'mobile', 'social']",discount-7-3-7
6,fafdcd668e3743c1bb461111dcafc2a4,discount,10,2,10,"['web', 'email', 'mobile', 'social']",discount-10-2-10
7,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,3,"['email', 'mobile', 'social']",informational-0-0-3
8,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,"['web', 'email', 'mobile', 'social']",bogo-5-5-5
9,2906b810c7d4411798c6938adc9daaa5,discount,10,2,7,"['web', 'email', 'mobile']",discount-10-2-7


In [98]:
#merge df_offer and df_completed
df_offer_completed=pd.merge(df_completed,df_offer,on="offer_id",how="left")
print(df_offer_completed.shape)
df_offer_completed.head()

(32070, 15)


,customer_id,gender,age_group,income_group,membership_year,event,time,offer_id,reward_x,offer_type,difficulty,reward_y,duration,channels,offer_key
0,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer completed,576,2906b810c7d4411798c6938adc9daaa5,2.0,discount,10,2,7,"['web', 'email', 'mobile']",discount-10-2-7
1,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer completed,414,f19421c1d4aa40978ebb69ca19b0e20d,5.0,bogo,5,5,5,"['web', 'email', 'mobile', 'social']",bogo-5-5-5
2,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer completed,528,fafdcd668e3743c1bb461111dcafc2a4,2.0,discount,10,2,10,"['web', 'email', 'mobile', 'social']",discount-10-2-10
3,0011e0d4e6b944f998e987f904e8c1e5,O,Middle Age Adult,Middle Income,2018,offer completed,576,0b1e1539f2cc45b7b9fa7c272da2e1d7,5.0,discount,20,5,10,"['web', 'email']",discount-20-5-10
4,0011e0d4e6b944f998e987f904e8c1e5,O,Middle Age Adult,Middle Income,2018,offer completed,252,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,discount,7,3,7,"['web', 'email', 'mobile', 'social']",discount-7-3-7


In [99]:
#create a new dataframe to analyze offer
df_offer_analyze=df_offer_completed[["customer_id","offer_id","channels","offer_key"]]
df_offer_analyze.head()


,customer_id,offer_id,channels,offer_key
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,"['web', 'email', 'mobile']",discount-10-2-7
1,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,"['web', 'email', 'mobile', 'social']",bogo-5-5-5
2,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,"['web', 'email', 'mobile', 'social']",discount-10-2-10
3,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,"['web', 'email']",discount-20-5-10
4,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,"['web', 'email', 'mobile', 'social']",discount-7-3-7


In [100]:
#most successful offer?
offer_key_completed=df_offer_analyze.offer_key.value_counts()
offer_key_completed

offer_key
discount-10-2-10    4957
discount-7-3-7      4847
bogo-5-5-7          4141
bogo-5-5-5          4074
discount-10-2-7     3860
bogo-10-10-7        3604
discount-20-5-10    3306
bogo-10-10-5        3281
Name: count, dtype: int64

In [101]:
#most successful channels?
channel_completed=df_offer_analyze.channels.value_counts()
channel_completed

channels
['web', 'email', 'mobile', 'social']    17159
['web', 'email', 'mobile']               8001
['email', 'mobile', 'social']            3604
['web', 'email']                         3306
Name: count, dtype: int64

In [102]:
#get dataset to analyze offer demogrgraphy
df_offer_demography=df_offer_completed[["customer_id","offer_key","age_group","gender","income_group"]]
print(df_offer_demography.shape)
df_offer_demography.head()

(32070, 5)


,customer_id,offer_key,age_group,gender,income_group
0,0009655768c64bdeb2e877511632db8f,discount-10-2-7,Young Adult,M,Middle Income
1,0009655768c64bdeb2e877511632db8f,bogo-5-5-5,Young Adult,M,Middle Income
2,0009655768c64bdeb2e877511632db8f,discount-10-2-10,Young Adult,M,Middle Income
3,0011e0d4e6b944f998e987f904e8c1e5,discount-20-5-10,Middle Age Adult,O,Middle Income
4,0011e0d4e6b944f998e987f904e8c1e5,discount-7-3-7,Middle Age Adult,O,Middle Income


In [103]:
#which offer older adult like most?
df_older_adult=df_offer_demography[df_offer_demography.age_group=="Older Adult"]
print(df_older_adult.shape)
df_older_adult.offer_key.value_counts()

(11898, 5)


offer_key
discount-10-2-10    1810
discount-7-3-7      1680
bogo-5-5-7          1540
bogo-5-5-5          1512
discount-10-2-7     1426
bogo-10-10-7        1387
discount-20-5-10    1294
bogo-10-10-5        1249
Name: count, dtype: int64

In [104]:
##which offer senior adult like most?
df_senior_adult=df_offer_demography[df_offer_demography.age_group=="Senior"]
print(df_senior_adult.shape)
df_senior_adult.offer_key.value_counts()

(7127, 5)


offer_key
discount-10-2-10    1077
discount-7-3-7      1054
bogo-5-5-7           911
discount-10-2-7      900
bogo-5-5-5           880
bogo-10-10-7         788
discount-20-5-10     759
bogo-10-10-5         758
Name: count, dtype: int64

In [105]:
#which offer female like most
df_female_offer=df_offer_demography[df_offer_demography.gender=="F"]
print(df_female_offer.shape)
df_female_offer.offer_key.value_counts()

(15316, 5)


offer_key
discount-10-2-10    2201
discount-7-3-7      2192
bogo-5-5-7          1979
bogo-5-5-5          1890
bogo-10-10-7        1834
discount-10-2-7     1830
bogo-10-10-5        1731
discount-20-5-10    1659
Name: count, dtype: int64

In [106]:
#which offer male like most
df_male_offer=df_offer_demography[df_offer_demography.gender=="M"]
print(df_male_offer.shape)
df_male_offer.offer_key.value_counts()

(16259, 5)


offer_key
discount-10-2-10    2691
discount-7-3-7      2588
bogo-5-5-5          2118
bogo-5-5-7          2090
discount-10-2-7     1971
bogo-10-10-7        1712
discount-20-5-10    1584
bogo-10-10-5        1505
Name: count, dtype: int64

In [107]:
#which offer middle income like most
df_income_offer=df_offer_demography[df_offer_demography.income_group=="Middle Income"]
print(df_income_offer.shape)
df_income_offer.offer_key.value_counts()

(19403, 5)


offer_key
discount-10-2-10    2977
discount-7-3-7      2959
bogo-5-5-7          2492
bogo-5-5-5          2474
discount-10-2-7     2362
bogo-10-10-7        2169
bogo-10-10-5        2002
discount-20-5-10    1968
Name: count, dtype: int64

In [108]:
#get df_received
print(df_received.shape)
df_received.head()

(66501, 3)


,customer_id,rcv_time,offer_id
0,0009655768c64bdeb2e877511632db8f,576,2906b810c7d4411798c6938adc9daaa5
2,0009655768c64bdeb2e877511632db8f,336,3f207df678b143eea3cee63160fa8bed
4,0009655768c64bdeb2e877511632db8f,168,5a8bc65990b245e5a138643cd4eb9837
6,0009655768c64bdeb2e877511632db8f,408,f19421c1d4aa40978ebb69ca19b0e20d
9,0009655768c64bdeb2e877511632db8f,504,fafdcd668e3743c1bb461111dcafc2a4


In [109]:
#link offer with df_receive
#merge df_offer and df_received
df_offer_received=pd.merge(df_received,df_offer,on="offer_id",how="left")
print(df_offer_received.shape)
df_offer_received.head()

(66501, 9)


,customer_id,rcv_time,offer_id,offer_type,difficulty,reward,duration,channels,offer_key
0,0009655768c64bdeb2e877511632db8f,576,2906b810c7d4411798c6938adc9daaa5,discount,10,2,7,"['web', 'email', 'mobile']",discount-10-2-7
1,0009655768c64bdeb2e877511632db8f,336,3f207df678b143eea3cee63160fa8bed,informational,0,0,4,"['web', 'email', 'mobile']",informational-0-0-4
2,0009655768c64bdeb2e877511632db8f,168,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,3,"['email', 'mobile', 'social']",informational-0-0-3
3,0009655768c64bdeb2e877511632db8f,408,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5,"['web', 'email', 'mobile', 'social']",bogo-5-5-5
4,0009655768c64bdeb2e877511632db8f,504,fafdcd668e3743c1bb461111dcafc2a4,discount,10,2,10,"['web', 'email', 'mobile', 'social']",discount-10-2-10


In [110]:
#which offer is most received?
offer_key_received=df_offer_received.offer_key.value_counts().sort_values(ascending=False)
offer_key_received

offer_key
discount-20-5-10       6726
bogo-5-5-7             6685
bogo-10-10-7           6683
informational-0-0-4    6657
discount-7-3-7         6655
discount-10-2-10       6652
informational-0-0-3    6643
discount-10-2-7        6631
bogo-10-10-5           6593
bogo-5-5-5             6576
Name: count, dtype: int64

In [111]:
#offer conversion rate
offer_key_converstion=(offer_key_completed/offer_key_received)*100
offer_key_converstion.sort_values(ascending=False)

offer_key
discount-10-2-10       74.518942
discount-7-3-7         72.832457
bogo-5-5-5             61.952555
bogo-5-5-7             61.944652
discount-10-2-7        58.211431
bogo-10-10-7           53.927877
bogo-10-10-5           49.764902
discount-20-5-10       49.152542
informational-0-0-3          NaN
informational-0-0-4          NaN
Name: count, dtype: float64

In [112]:
#which channel is used most for providing offer?
channel_received=df_offer_received.channels.value_counts().sort_values(ascending=False)
channel_received

channels
['web', 'email', 'mobile', 'social']    26476
['web', 'email', 'mobile']              19973
['email', 'mobile', 'social']           13326
['web', 'email']                         6726
Name: count, dtype: int64

In [113]:
#channel conversion rate
channel_converstion=(channel_completed/channel_received)*100
channel_converstion.sort_values(ascending=False)

channels
['web', 'email', 'mobile', 'social']    64.809639
['web', 'email']                        49.152542
['web', 'email', 'mobile']              40.059080
['email', 'mobile', 'social']           27.044875
Name: count, dtype: float64

In [114]:
#which reward is most received
df_offer_received['reward'] = df_offer_received['reward'].astype('Int64')
reward_received=df_offer_received.reward.value_counts().sort_values(ascending=False)
reward_received

reward
5     19987
0     13300
2     13283
10    13276
3      6655
Name: count, dtype: Int64

In [115]:
#which reward is most completed
df_completed['reward'] = df_completed['reward'].astype('Int64')
reward_completed=df_completed.reward.value_counts().sort_values(ascending=False)
reward_completed

reward
5     11521
2      8817
10     6885
3      4847
Name: count, dtype: Int64

In [116]:
#reward conversion
reward_conversion=(reward_completed/reward_received)*100
reward_conversion.sort_values(ascending=False)

reward
3     72.832457
2     66.378077
5     57.642468
10      51.8605
0          <NA>
Name: count, dtype: Float64

In [117]:
#overall how many transaction by each customer
transaction_counts=df_transaction.customer_id.value_counts()
transaction_counts

customer_id
79d9d4f86aca4bed9290350fb43817c2    36
8dbfa485249f409aa223a2130f40634a    36
94de646f7b6041228ca7dec82adb97d2    35
5e60c6aa3b834e44b822ea43a3efea26    32
b1f4ece7d49342628a9ed77aee2cde58    32
                                    ..
4f4777c7c22648f58098e89a34426e07     1
f8eff6816be34191b76e9b4614ce5224     1
4eeaab922a754d58a14733cfd2952279     1
0f628df0d3fb4e1192f156de98ec4a0b     1
f904dad31185443c909f7722c28e9840     1
Name: count, Length: 14492, dtype: int64

In [118]:
# Count how many customers made a specific number of transactions
customer_frequency = transaction_counts.value_counts().sort_index(ascending=False)

print(customer_frequency)

count
36       2
35       1
32       2
31       1
30       5
29       7
28       7
27      11
26      15
25      39
24      35
23      62
22      99
21     113
20     133
19     176
18     240
17     266
16     337
15     375
14     472
13     573
12     643
11     744
10     888
9      864
8     1135
7     1174
6     1325
5     1370
4     1331
3     1054
2      674
1      319
Name: count, dtype: int64


In [119]:
#how many offer_completed by each customer
completion_counts=df_completed.customer_id.value_counts()
completion_counts

customer_id
eab78e2cf26f4af494d9535ac3835f00    6
9b42f9e0dc2e465982845e31f0b5c02d    6
1948b3c194c542d0a2a084a558f31834    6
6e92f1561dab4dc180245974f546f5b5    6
9b9858f0a599424688bc2c3d0d2a6d3b    6
                                   ..
85ba195ec59d433e81b89e72803b11b0    1
85a4b98da3254e0986fc5a1ee9d5c1cc    1
8583bb774d2e4496a96c1550a036c8b8    1
856f018e2ae94741a673c3acbd9e1b88    1
8569444509e1462abe0310cbbd323a86    1
Name: count, Length: 11986, dtype: int64

In [120]:
# Count how many customers made a specific number of completion
customer_frequency = completion_counts.value_counts().sort_index(ascending=False)

print(customer_frequency)

count
6     178
5     919
4    2156
3    2991
2    3068
1    2674
Name: count, dtype: int64


In [121]:
#total transaction amount after completion
total_trans_amount=df_offer_tran.amount.sum()
total_trans_amount

np.float64(606355.75)

In [122]:
#total reward amount after completion
total_reward_amount=df_completed.reward.sum()
total_reward_amount

np.int64(158630)

In [123]:
#reward sale ratio
reward_sale_perc=(total_reward_amount/total_trans_amount)*100
reward_sale_perc

np.float64(26.161209817833836)

In [130]:
#offer related average transaction
df_offer_trans_total=df_offer_tran.amount.sum()
df_offer_trans_total/df_offer_tran.shape[0]

np.float64(20.498149149792095)

In [137]:
#non offer related average transaction

non_offer_trans=df_transaction.shape[0]-df_offer_tran.shape[0]
non_offer_trans_amount=df_transaction.amount.sum()-df_offer_tran.amount.sum()
non_offer_trans_amount/non_offer_trans

np.float64(11.958407328134273)

In [138]:
#how many transaction is linked to offers
df_offer_tran.shape[0]/df_transaction.shape[0]

0.23863920553095025

In [140]:
#analyze young adult
df_young_adult=df_completed[df_completed.age_group=="Young Adult"]
df_young_adult.head()

,customer_id,gender,age_group,income_group,membership_year,event,time,offer_id,reward
1,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer completed,576,2906b810c7d4411798c6938adc9daaa5,2
7,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer completed,414,f19421c1d4aa40978ebb69ca19b0e20d,5
10,0009655768c64bdeb2e877511632db8f,M,Young Adult,Middle Income,2017,offer completed,528,fafdcd668e3743c1bb461111dcafc2a4,2
59,0020ccbbb6d84e358d3414a3ff76cffd,F,Young Adult,Middle Income,2016,offer completed,222,2298d6c36e964ae4a3e7e9706d1fb8c2,3
64,0020ccbbb6d84e358d3414a3ff76cffd,F,Young Adult,Middle Income,2016,offer completed,600,9b98b8c7a33c4b65b9aebfe6a799e6d9,5


In [141]:
df_young_adult.income_group.value_counts()

income_group
Middle Income    2599
Low Income       1290
High Income         0
Name: count, dtype: int64

In [142]:
df_young_adult.offer_id.value_counts()

offer_id
2298d6c36e964ae4a3e7e9706d1fb8c2    686
fafdcd668e3743c1bb461111dcafc2a4    673
f19421c1d4aa40978ebb69ca19b0e20d    534
9b98b8c7a33c4b65b9aebfe6a799e6d9    526
2906b810c7d4411798c6938adc9daaa5    452
ae264e3637204a6fb9bb56bc8210ddfd    357
0b1e1539f2cc45b7b9fa7c272da2e1d7    334
4d5c57ea9a6940dd891ad53e9dbe8da0    327
Name: count, dtype: int64

2298d6c36e964ae4a3e7e9706d1fb8c2  is discount 7-3-7. fafdcd668e3743c1bb461111dcafc2a4 discount-10-2-10